In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
nl = 2 # number of lobbyists
b = 500 # [30, 60, 150, 300, 500, 750, 1000]  # lobbyists budget in the simulation
basepath = f'../results/sensitivity_budgets/{nl}_lobbyists/{b}k_budget'
filename = 'config.json'
with open(os.path.join(basepath, filename), 'r') as f:
    params = json.load(f)

In [ ]:
print(params)

In [ ]:
p_o = params['p_o']
p_p = params['p_p']
lambda_values = params['lambda_values']
phi_values = params['phi_values']
n_lobbyists = params['n_lobbyists']
nruns = params['nruns']
lobbyists_data = params['lobbyists_data']

In [ ]:
kinds = ['weights', 'probabilities']

data = []

for folder in os.listdir(basepath):
    paramspath = os.path.join(basepath, folder)
    try:
        lambda_v = float(folder.split('_')[0])  
        phi_v = float(folder.split('_')[1])
    except:
        continue
    
    for kind in kinds:
        if os.path.exists(paramspath+f'/{kind}_average_metrics.json'):
            filename = os.path.join(paramspath, f'{kind}_average_metrics.json') 
            with open(filename, 'r') as f:
                avg_metrics = json.load(f)
                            
            columns = ['kind', 'n_lobbyists', 'lambda', 'phi', 'nruns', 'p_o', 'p_p']
            values = [kind, n_lobbyists, lambda_v, phi_v, nruns, p_o, p_p]
            
            for metric_name in avg_metrics.keys():
                if metric_name == 'average_opinions':
                    avg = avg_metrics[metric_name]['avg']
                    std = avg_metrics[metric_name]['std']
                    z_stat = avg_metrics[metric_name]['z_statistic']
                    p_val = avg_metrics[metric_name]['p_value']
                    columns.extend([f'avg_{metric_name}', f'std_{metric_name}', f'z_stat_{metric_name}', f'p_val_{metric_name}'])
                    values.extend([avg, std, z_stat, p_val])
                elif metric_name != 'lobbyists_performance':
                    avg = avg_metrics[metric_name]['avg']
                    std = avg_metrics[metric_name]['std']
                    columns.extend([f'avg_{metric_name}', f'std_{metric_name}'])
                    values.extend([avg, std])
                else:
                    for id in lobbyists_data.keys():
                        if lobbyists_data[id]['m'] == 0:
                            print('entering here')
                            avg = avg_metrics[metric_name][id]['avg']
                            std = avg_metrics[metric_name][id]['std']
                            columns.extend([f'avg_{metric_name}_pess_{id}', f'std_{metric_name}_pess_{id}'])
                            values.extend([avg, std])
                            break
                    for id in lobbyists_data.keys():
                        if lobbyists_data[id]['m'] == 1:
                            print('entering here')
                            avg = avg_metrics[metric_name][id]['avg']
                            std = avg_metrics[metric_name][id]['std']
                            columns.extend([f'avg_{metric_name}_opt_{id}', f'std_{metric_name}_opt_{id}'])
                            values.extend([avg, std])
                            break
                                    
            row = dict(zip(columns, values))
                
            data.append(row)      

df = pd.DataFrame(data)
datapath = os.path.join(basepath, 'aggregate_metrics.csv')        
df.to_csv(datapath)

In [ ]:
import seaborn as sns

def heatmap(kind, metric, figname=None):
    # Filter the DataFrame
    
    df_filtered = df[df["kind"] == kind]

    # Pivot the table to have phi and lambda as axes
    heatmap_data = df_filtered.pivot(index="lambda", columns="phi", values=metric)

    # Plot the heatmap
    plt.figure(figsize=(8, 6))
    if metric == 'avg_average_opinions':
        sns.heatmap(heatmap_data, vmin=0, vmax=1, cmap="Blues", annot=True, fmt=".2f")
    elif metric == 'avg_number_iterations':
        sns.heatmap(heatmap_data, cmap="Blues", annot=True, fmt=".0f")
    elif metric.startswith('avg_lobbyists_performance') or metric == 'avg_average_pairwise_distance' or metric == 'avg_std_opinions':
        sns.heatmap(heatmap_data, vmin = 0, cmap="Blues", annot=True, fmt=".2f")
    elif metric == 'avg_effective_number_clusters':
        sns.heatmap(heatmap_data, vmin = 1,vmax = 3, cmap="Blues", annot=True, fmt=".2f")
    else:
        sns.heatmap(heatmap_data, cmap="Blues", annot=True, fmt=".2f")
    plt.title(f"Heatmap of {metric} for {kind}")
    plt.xlabel("Phi")
    plt.ylabel("Lambda")
    figpath = os.path.join(basepath, 'figures')
    os.makedirs(figpath, exist_ok=True)
    
    if figname is not None:
        figname = os.path.join(figpath, figname)
        plt.savefig(figname, dpi=300, bbox_inches='tight', facecolor='white')
    else:
        plt.show()
        
    plt.close()

for kind in ['probabilities', 'weights']:
    if kind in df['kind'].unique() :
        for col in columns:
            if col.startswith('avg') or col.startswith('z_stat') or col.startswith('p_val') :
                metric = col
                # heatmap(kind, metric, figname=f'{kind}_{metric}.png')
                figpath = os.path.join(basepath, 'figures')
                os.makedirs(figpath, exist_ok=True)
                figname = os.path.join(figpath, f'heatmap_{kind}_{metric}.png')
                heatmap(kind, metric, figname=figname)
                heatmap(kind, metric, figname=None)

In [ ]:
kinds = ['weights', 'probabilities']

data = []

for folder in os.listdir(basepath):
    paramspath = os.path.join(basepath, folder)
    try:
        lambda_v = float(folder.split('_')[0])  
        phi_v = float(folder.split('_')[1])
    except:
        continue
    
    for kind in kinds:
        if os.path.exists(paramspath+f'/{kind}_average_metrics.json'):
            filename = os.path.join(paramspath, f'{kind}_metrics_distributions.json') 
            with open(filename, 'r') as f:
                metrics = json.load(f)
                           
            columns = ['kind', 'lambda', 'phi']
            values = [kind, lambda_v, phi_v]
            
            for metric_name in metrics.keys():
                if metric_name != 'lobbyists_performance':
                    columns.append(metric_name)
                    values.append(metrics[metric_name])
                else:
                    for id in lobbyists_data.keys():
                        if lobbyists_data[id]['m'] == 0:
                            columns.append(f'{metric_name}_pess_{id}')
                            values.append(metrics[metric_name][id])
                            break
                    for id in lobbyists_data.keys():
                        if lobbyists_data[id]['m'] == 1:
                            columns.append(f'{metric_name}_opt_{id}')
                            values.append(metrics[metric_name][id])
                            break
                    #for id in lobbyists_data.keys():
                    #   columns.append(f'{metric_name}_{id}')
                    #   values.append(metrics[metric_name][id])
                                
            row = dict(zip(columns, values))
                
            data.append(row)
        
df = pd.DataFrame(data)
datapath = os.path.join(basepath, 'metrics_distributions.csv')
df.to_csv(datapath)

In [ ]:
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def explode_lists(df):
    """Expands list values into separate rows while keeping other columns unchanged."""
    id_vars = ["kind", "lambda", "phi"]
    value_vars = [col for col in df.columns if col not in id_vars]
    
    exploded_data = []
    for _, row in df.iterrows():
        max_len = max(len(row[col]) for col in value_vars)
        for i in range(max_len):
            exploded_row = {col: row[col] if col in id_vars else row[col][i] for col in df.columns}
            exploded_data.append(exploded_row)
    
    return pd.DataFrame(exploded_data)

def plot_metrics_grid(df):
    df = explode_lists(df)
    metrics = [col for col in df.columns if col not in ["kind", "lambda", "phi"]]
    lambda_values = sorted(df["lambda"].unique())
    num_lambdas = len(lambda_values)
    
    kind = 'probabilities'
    df_kind = df[df["kind"] == kind]
    for metric in metrics:
        fig, axes = plt.subplots(1, num_lambdas, figsize=(5 * num_lambdas, 5), sharey=True)
        
        if num_lambdas == 1:
            axes = [axes]  # Ensure axes is iterable if only one lambda value exists
        
        for i, lambd in enumerate(lambda_values):
            ax = axes[i]
            subset = df_kind[df_kind["lambda"] == lambd]
            
            sns.boxplot(x="phi", y=metric, data=subset, ax=ax)
            ax.set_title(f"{metric} lambda={lambd}")
            ax.set_xlabel("Phi")
            ax.set_ylabel(metric)
        
        plt.tight_layout()
        figpath = os.path.join(basepath, 'figures')
        os.makedirs(figpath, exist_ok=True)
        figname = os.path.join(figpath, f'boxplots_{metric}.png')
        plt.savefig(figname, dpi=300, bbox_inches='tight', facecolor='white')
        plt.show()
        plt.close()

# Example usage
#df = pd.read_csv(datapath)  # Load the DataFrame from CSV or other sources
plot_metrics_grid(df)
